In [ ]:
USE_PRIVATE_DISTRO = True

if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU /content/drive/MyDrive/SMC\ 10/DDSP-10/dist/ddsp-1.2.0.tar.gz
else:
    !pip install -qU ddsp

%tensorflow_version 2.x
import tensorflow as tf

from ddsp.synths import AmplitudeModulation, FrequencyModulation
from ddsp.colab.colab_utils import specplot
from ddsp import core

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%config InlineBackend.figure_format='retina'

import numpy as np
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output

# FM Synth

In [ ]:
SAMPLE_RATE = 48000
DURATION = 2

N_SAMPLES = SAMPLE_RATE * DURATION

###Definition

In [ ]:
fm_synth = FrequencyModulation(n_samples=N_SAMPLES,
                               sample_rate=SAMPLE_RATE,
                               add_endpoint=False)

fade = np.linspace(0,1,1000)

def synthesize (f0, a1, i1, a2, i2, a3, i3, a4, i4, 
                m21, m31, m32, m41, m42, m43):
  
  audio = fm_synth.get_signal([[[f0]]],
                              [[[a1, i1]+get_random_ADSR(),[a1, i1]+get_random_ADSR()]], 
                              [[[a2, i2]+get_random_ADSR(),[a2, i2]+get_random_ADSR()]], 
                              [[[a3, i3]+get_random_ADSR(),[a3, i3]+get_random_ADSR()]], 
                              [[[a4, i4]+get_random_ADSR(),[a4, i4]+get_random_ADSR()]],
                              [[[m21, m31, m32, m41, m42, m43]]],
                              )

  audio /= tf.reduce_max(audio[0,:], axis=0)*1.5
  audio = audio[0].numpy()
  audio[:fade.size] = audio[:fade.size] * fade
  audio[-fade.size:] = audio[-fade.size:] * (1-fade)

  return audio

def render (f0, a1, i1, a2, i2, a3, i3, a4, i4, 
            m21, m31, m32, m41, m42, m43,
            sample_rate=SAMPLE_RATE,
            autoplay=False,
            context=None):
  
  audio = synthesize(f0, a1, i1, a2, i2, a3, i3, a4, i4, 
                m21, m31, m32, m41, m42, m43)
  if context is None:
    display(Audio(audio, rate=SAMPLE_RATE, autoplay=autoplay))
  else:
    with context:
      clear_output(True)
      display(Audio(audio, rate=SAMPLE_RATE, autoplay=autoplay))

def get_random_ADSR(n_samples = N_SAMPLES):
  env = np.random.random(1).tolist()
  return env

### Test scaling

In [ ]:
fm_no_scaled = FrequencyModulation(n_samples=N_SAMPLES,
                                   sample_rate=SAMPLE_RATE,
                                   amp_scale_fn=None)

fm_scaled = FrequencyModulation(n_samples=N_SAMPLES,
                                sample_rate=SAMPLE_RATE,
                                index_scale=True,
                                ar_scale=True)
f0=[[[440]]]
op1=np.array([[[1, -10]+get_random_ADSR()]])
op2=np.array([[[2, 12]+get_random_ADSR()]])
op3=np.array([[[3, 13]+get_random_ADSR()]])
op4=np.array([[[4, 14]+get_random_ADSR()]])
m=np.array([[[.21, .931, 32, 41, 42, 43]]])

controls = fm_no_scaled.get_controls(f0, op1, op2, op3, op4, m)
for key in controls:
  print(key, controls[key])

controls = fm_scaled.get_controls(f0, op1, op2, op3, op4, m)
for key in controls:
  print(key, controls[key])

###GUI

In [ ]:
current_algorithm = 4

algorithms = [[1,0,0,0], [1,0,0,0], [1,0,0,0], [1,0,0,0], [1,0,0,0],
              [1,1,0,0], [1,1,0,0], [1,1,0,0], [1,1,0,0], [1,1,0,0], 
              [1,1,1,0], [1,1,1,0], [1,1,1,0],
              [1,1,1,1]] 

connections=[[43,32,21],[42,32,21],[42,31,21],[43,42,31,21],[41,31,21],
             [42,31],[42,32],[42,32,31],[43,32],[43,32,31],
             [43],[43,42],[43,42,41],
             []]

mods = {'21':0, '31':0, '32':0, '41':0, '42':0, '43':0}
modulators = {"4":[], "3":[], "2":[]}
indexes = []
voices = []



def w_go_clicked(b):
  global mods

  for f in range(4):
    for g in range(f):
      if (f+1)*10+g+1 in connections[current_algorithm]:
        mods[str((f+1)*10+g+1)] = modulators[str(f+1)][g*2+1].value
      else:
        mods[str((f+1)*10+g+1)] = 0

  render(440, 
         algorithms[current_algorithm][0], float(indexes[0].value), 
         algorithms[current_algorithm][1], float(indexes[1].value), 
         algorithms[current_algorithm][2], float(indexes[2].value), 
         algorithms[current_algorithm][3], float(indexes[3].value), 
         mods['21'], mods['31'], mods['32'], 
         mods['41'], mods['42'], mods['43'],
         autoplay = True, context=wave_output)
  # if spectrogram:
  #   specplot(audio)
w_go = widgets.Button(
    description='Synthesize audio'
)
w_go.on_click(w_go_clicked)



def w_rand_clicked(b):
  global mods

  for f in range(4):
    if f==1:
      indexes[f-1].value = np.random.randint(1,4)*0.5
    else:
      indexes[f-1].value = np.random.randint(1,24)*0.5
    for g in range(f):
      modulators[str(f+1)][g*2+1].value = np.random.randint(0,12)
      if (f+1)*10+g+1 in connections[current_algorithm]:
        mods[str((f+1)*10+g+1)] = modulators[str(f+1)][g*2+1].value
      else:
        mods[str((f+1)*10+g+1)] = 0

  render(440, 
         algorithms[current_algorithm][0], float(indexes[0].value), 
         algorithms[current_algorithm][1], float(indexes[1].value), 
         algorithms[current_algorithm][2], float(indexes[2].value), 
         algorithms[current_algorithm][3], float(indexes[3].value), 
         mods['21'], mods['31'], mods['32'], 
         mods['41'], mods['42'], mods['43'],
         autoplay = True, context=wave_output)
  # if spectrogram:
  #   specplot(audio)
w_rand = widgets.Button(
    description='Randomize!!!',
    button_style='danger'
)
w_rand.on_click(w_rand_clicked)



def w_alg_changed(change):
  global current_algorithm

  if change['type'] == 'change' and change['name'] == 'value':
    current_algorithm = w_alg.index
    refresh_gui()

w_alg = widgets.Dropdown(
    options=[('1   Series', 0), ('2', 1), ('3', 2), 
            ('4', 3), ('5', 4), ('6', 5),
            ('7', 6), ('8', 7), ('9', 8),
            ('10   1-to-3', 9), ('11   Parallel', 10), ('12   3-to-1', 11),
            ('?1', 12),('?2', 13)],
    value=current_algorithm,
    description='Algorithm:',
)

w_alg.observe(w_alg_changed)



def refresh_gui():
  for f in range(4):
    voices[f].value=algorithms[current_algorithm][f]==1
    for g in range(f):
      if (f+1)*10+g+1 in connections[current_algorithm]:
        # modulators[str(f+1)][g*2+1].disabled = False
        modulators[str(f+1)][g*2].layout = layout=widgets.Layout(visibility='visible')
        modulators[str(f+1)][g*2+1].layout = layout=widgets.Layout(visibility='visible', width='15%')
      else:
        # modulators[str(f+1)][g*2+1].disabled = True
        modulators[str(f+1)][g*2].layout = layout=widgets.Layout(visibility='hidden')
        modulators[str(f+1)][g*2+1].layout = layout=widgets.Layout(visibility='hidden', width='15%')


for f in range(1,5):

  voices.append(
      widgets.Checkbox(
        value=0,
        indent=False,
        disabled=True,
        layout=widgets.Layout(width='5%'))
      )

  indexes.append(
      widgets.FloatSlider(
        value=1.0, min=0.25, max=12.0, step=0.25,
        layout=widgets.Layout(width='15%'))
      )

  if (str(f) in modulators):  
    for g in range(1,f):
      modulators[str(f)].append(
        widgets.Label("%d→%d" % (f, g))
      )
      modulators[str(f)].append(
        widgets.FloatSlider( 
        value=1.0, min=0, max=12.0, step=0.5,
        layout=widgets.Layout(width='15%'))
      )



gui_output = widgets.Output(layout={'border': '1px solid #999999', 
                             'padding': '15px',
                             'margin': '15px'})

wave_output = widgets.Output(layout={'padding': '15px 0 0 0'})



with gui_output:
  display(w_alg)
  for f in range (3,-1,-1):
    elements = [ widgets.Label('OP'+str(f+1)), voices[f], widgets.Label('Idx'), indexes[f]]
    if str(f+1) in modulators:
      elements = elements + modulators[str(f+1)]
    display(widgets.HBox(elements))
  
  refresh_gui()
  display(widgets.HBox([w_go, w_rand]))
  display(wave_output)


### Synth

In [ ]:
display(gui_output)

### One op

In [ ]:
print("Single op")
render(440, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

print("Single op, index 3")
render(440, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


### Two ops

In [ ]:
print("Ops in parallel, indexes 3 and 0.5")
render(440, 1, 3, 1, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

# 2 -> 1
print("Ops in series 2 -> 1")
render(440, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0)

print("Ops in series 2 -> 1")
render(440, 1, 1, 0, 0.5, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0)

print("Ops in series 2 -> 1")
render(440, 1, 1, 0, 0.01, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0)
#-----------------------------OP2 level = 98


### Three ops

In [ ]:
print("Ops in parallel, indexes 0.5, 3 and 6 - Alg 25")
render(440, 1, 0.5, 1, 3, 1, 6, 0, 0, 0, 0, 0, 0, 0, 0)

print("Alg 23")
render(440, 1, 0.5, 1, 1, 0, 1, 0, 0, 0, 0, 12, 0, 0, 0)

print("Alg 3")
render(440, 1, 0.5, 0, 1, 0, 1, 0, 0, 11.5, 0, 12.5, 0, 0, 0)

print("Alg 16")
render(440, 1, 0.5, 0, 1, 0, 1, 0, 0, 11.5, 12.5, 0, 0, 0, 0)

print("Alg 21")
render(440, 1, 0.5, 1, 1, 0, 1, 0, 0, 0, 12.5, 12.5, 0, 0, 0)

### Four ops

In [ ]:
print("Alg 2")
render(440, 1, 1, 0, 1.01, 0, 0.5, 0, 3, 12, 0, 11, 0, 0, 11)

In [ ]:
raise SystemExit("Stop right there!")

# AM Synth

In [ ]:
SAMPLE_RATE = 48000
DURATION = 4

N_SAMPLES = SAMPLE_RATE * DURATION

am_synth = AmplitudeModulation(n_samples=N_SAMPLES,
                               sample_rate=SAMPLE_RATE)

###Single frame

In [ ]:
car_amp = np.ones(1)[np.newaxis, :, np.newaxis]
car_freq = 440.0 * np.ones(1)[np.newaxis, :, np.newaxis]

mod_amp = 2 * np.ones(1)[np.newaxis, :, np.newaxis]
mod_freq = 0.5 * np.ones(1)[np.newaxis, :, np.newaxis]

audio = am_synth.get_signal(car_amp, car_freq, mod_amp, mod_freq)
audio /= tf.reduce_max(audio[0,:], axis=0).numpy();

display(Audio(audio, rate=SAMPLE_RATE, autoplay=False))
specplot(audio)

### Multiple frames

In [ ]:
FRAME_RATE = 250
TIME_STEPS = FRAME_RATE * DURATION


car_amp = np.concatenate((np.linspace(-10,1,100), np.ones(TIME_STEPS-100)), axis=0)
car_amp = car_amp[np.newaxis, :, np.newaxis]

car_freq = 440.0 * np.ones(TIME_STEPS)[np.newaxis, :, np.newaxis]

mod_amp = np.linspace(-2,1,TIME_STEPS)[np.newaxis, :, np.newaxis]

mod_freq = np.concatenate((2.0*np.ones(TIME_STEPS//2), np.linspace(2,50,TIME_STEPS//2)), axis=0)[np.newaxis, :, np.newaxis]

audio = am_synth.get_signal(car_amp, car_freq, mod_amp, mod_freq)
audio /= tf.reduce_max(audio[0,:], axis=0).numpy();

display(Audio(audio, rate=SAMPLE_RATE, autoplay=False))
specplot(audio)

f, ax = plt.subplots(1, 2, figsize=(10.5, 3))
ax[0].plot(car_amp[0,:,0])
ax[0].plot(mod_amp[0,:,0])
ax[0].set_ylabel('Amplitude')
ax[0].set_xlabel('Frames')
ax[0].legend(['Carrier', 'LFO'])
ax[1].plot(mod_freq[0,:,0])
ax[1].set_ylabel('Frequency (Hz)')
ax[1].set_xlabel('Frames')
_=ax[1].legend(['Carrier', 'LFO'])

In [ ]:
raise SystemExit("Stop right there!")